# LLMs and RAG with DataChain

In LLM applications nowadays, the emerging standard pattern for most use-cases is to employ a pre-trained model with an API from a 3rd party provider and to augment it with a RAG context. Naive application of "latest and greatest" models with no prompt engineering, testing and evaluation of RAG context can lead to needlessly expensive operational costs at best and dissapointingly poor performance at worst.

Therefore, just like in machine learning training, we need to version all that data as we finetune our applications to be able to correctly evaluate the effect of any changes we apply to our models. We can experiment with the LLM choice, prompt engineering, the way we process data for our RAG context (pre-processing, embedding, ...) and so on.

In this example, we will see how we can use DataChain to create such a controlled development environment and how it can help us when we evaluate any fine-tuning of our LLM applications.

We will see how to use DataChain to version our RAG context datasets to preserve reproducibility of our fine-tuning experiments as the RAG context changes. We will also see how to use DataChain in the evaluation of fine-tuning by comparing two different text embedding models and saving (and versioning) the results with additional context.

## Processing a large collection of documents

Let's say that we have a collection of relevant documents which we want to use as context in LLM queries in our chatbot application. We will be using DataChain to create, store and version vector embeddings of our documents.

In this example we will be using papers from the [Neural Information Processing Systems](https://papers.neurips.cc/paper/) conference. 

We will proceed in the following steps:
1. [Data ingestion with DataChain](#data-ingestion) - we will use DataChain to ingest the data, taking advantage of its lazy evaluation feature to only ingest the data we need
1. [Data processing with the Unstructured Python library](#processing-the-documents-individually)
1. [Scaling the data processing with DataChain](#processing-the-documents-at-scale-using-datachain-udfs)
1. [Using Datachain to evaluate different embedding models](#evaluation)
1. [Adding extra context by combining datasets](#adding-more-context---merging-datasets)

In [1]:
from sqlalchemy import func
from sqlalchemy import cast

from copy import deepcopy
from typing import Optional
from collections.abc import Iterator

from datachain.lib.dc import DataChain, C
from datachain.lib.data_model import DataModel
from datachain.lib.file import File
from datachain.sql.functions.array import cosine_distance

from unstructured.partition.pdf import partition_pdf
from unstructured.chunking.title import chunk_by_title

from unstructured.embed.huggingface import HuggingFaceEmbeddingConfig, HuggingFaceEmbeddingEncoder

### Data ingestion

We will first ingest the dataset. The data are saved on a cloud storage, so we use the `.from_storage` DataChain method. We will also use the `.filter` method to restrict ourselves only to `.pdf` files (the storage contains many other data which we do not need).

Notice that:

1. Since DataChain employs lazy evaluation, no data are actually loaded just yet (until we invoke an action such as showing or saving our DataChain)
1. The previous point also means that when we filter out all non-pdf files, DataChain doesn't actually waste time loading their content only to throw them away later. This makes DataChain a lot more scalable than tools with eager evaluation.
1. The `.from_storage` method of DataChain operates on the level of the entire bucket. This means that even if the files are stored using a complicated directory structure and potentially uploaded irregularly into this structure, we can retrieve or update our DataChain of articles with just a simple one-line command

In [2]:
dc_papers = (
    DataChain.from_storage("gs://datachain-demo/neurips")
    .filter(C.name.glob("*.pdf"))
    )

In [3]:
dc_papers.show(3)

Listing gs://datachain-demo: 738 objects [00:00, 871.29 objects/s]
Processed: 738 rows [00:00, 20346.78 rows/s]


,file,file,file,file,file,file,file,file,file,file
,source,parent,name,size,version,etag,is_latest,last_modified,location,vtype
0,gs://datachain-demo,neurips/1987/file,02e74f10e0327ad868d138f2b4fdd6f0-Paper.pdf,2291566,1721047139405563,CPudi5uIqYcDEAE=,1,2024-07-15 12:38:59.443000+00:00,None,
1,gs://datachain-demo,neurips/1987/file,03afdbd66e7929b125f8597834fa83a4-Paper.pdf,1322648,1721047138865046,CJaf6pqIqYcDEAE=,1,2024-07-15 12:38:58.917000+00:00,None,
2,gs://datachain-demo,neurips/1987/file,072b030ba126b2f4b2374f342be9ed44-Paper.pdf,1220711,1721046993295769,CJmztdWHqYcDEAE=,1,2024-07-15 12:36:33.340000+00:00,None,



[Limited by 3 rows]


DataChain created a record for each `pdf` file in the `neurips` directory, generating a `file` signal for each file. The file signal contains subsignals with metadata about each file, like `file.name` and `file.size`. Aggregate signals like `file` that contain multiple subsignals are called features.

You can use the `file` feature to not only get metadata about each file, but also to open and read the file as needed.

### Processing the documents individually

We now want to ingest the content of the pdf files as text, divide it into chunks and vectorize those for our RAG application. We are interested in comparing two different models for embeddings. Normally, we would also do some pre-processing and cleaning of the text before vectorization, but we will skip it here for brevity.

We will first do all this with an example of a single pdf using the `unstructured` Python library and then we will see how we can scale this up to the entire bucket with the help of DataChain.

First, we ingest and partition the pdf file and chunk it.


In [4]:
chunks = chunk_by_title(partition_pdf(filename="sample.pdf"))


Next, we vectorize each chunk using HuggingFace embedding encoders. Ideally, we want the smallest model possible while maintaining accuracy to increase speed and reduce costs of embeddings. We will see how embeddings from a candidate model `MODEL_NEW` differ from embeddings produced by the existing model `MODEL_OLD`.

In [5]:
MODEL_NEW = "sentence-transformers/paraphrase-MiniLM-L6-v2" 
MODEL_OLD = "sentence-transformers/all-MiniLM-L6-v2"

embedding_encoder_new = HuggingFaceEmbeddingEncoder(
     config=HuggingFaceEmbeddingConfig(model_name=MODEL_NEW, model_kwargs={"device":"cuda"}, encode_kwargs={"normalize_embeddings":True})
)

chunks_embedded_new = embedding_encoder_new.embed_documents(chunks)

embedding_encoder_old = HuggingFaceEmbeddingEncoder(
     config=HuggingFaceEmbeddingConfig(model_name=MODEL_OLD, model_kwargs={"device":"cuda"}, encode_kwargs={"normalize_embeddings":True})
)

# we need deepcopy here because unstructured creates lists of references to elements
chunks_embedded_old = embedding_encoder_old.embed_documents(deepcopy(chunks))

/home/tibor/Repos/datachain/.venv/lib64/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


We now have our chunks vectorized and ready for comparison (e.g. with cosine similarity). However, we are missing a few ingredients:

1. ***Scaling*** - we only processed a single pdf file and we had to manually specify its path. We need to find a way to process all our documents at scale instead and to save the results.
2. ***Saving and Versioning*** - even if we only had a single or a few PDF files we would like to use in our RAG, it is a good practice to version the outputs so that we can keep track of and fine-tune our RAG application. If we simply save the current results to a bucket and overwrite it each time the source is updated, we lose this. We could version the results manually, e.g. by adding a timestamp to the blob name, but that is not very reliable and will lead to unnecessary copies of files.

### Processing the documents at scale, using DataChain UDFs

We will now use DataChain to solve the scaling and versioning issues we outline above. We will create a DataChain user-defined function (UDF) to process all our PDF files the way we did above with a single file (without us having to manually provide file paths) and save the outputs in a Datachain.

The DataChain UDF functionality will allow us to generate additonal columns in our DataChain, iterating over each of the files listed in it.

We first need to define a DataModel class, which will define the types of our outputs. Inputs and outputs need to be specified like this when we use custom functinos in Datachain.

In [6]:
# Define the output as a Feature class
class Chunk(DataModel):
    key: str
    text: str
    embeddings_new: list[float]
    embeddings_old: list[float]

In the above we define `Chunk` by specifying the names and types of new columns on the output.

We then define our processing function `pdf_chuks`:

In [7]:
# Use signatures to define input/output types (these can be Feature or regular Python types)
def pdf_chunks(file: File) -> Iterator[Chunk]:
    # Ingest the file
    with file.open() as f:
        chunks = partition_pdf(file=f)

    chunks_embedded_new = embedding_encoder_new.embed_documents(chunks)
    chunks_embedded_old = embedding_encoder_old.embed_documents(deepcopy(chunks))

    # Add new rows to DataChain
    for chunk, chunk_orig in zip(chunks_embedded_new, chunks_embedded_old):
        record = {}
        record["key"] = file.name.removesuffix("-Paper.pdf")
        record["text"] = chunk.text
        record["embeddings_new"] = chunk.embeddings
        record["embeddings_old"] = chunk_orig.embeddings

        yield Chunk(**record)

Here, the syntax is the same as with any other Python function, except that we specify the input and output types using type hints

```
def pdf_chunks(file: File) -> Iterator[Chunk]:
```
Here, `file` specifies that we pass all `file` columns of the original dataset on the input and `Iterator[Chunk]` specifies that we get a bunch of `Chunk` rows on the output (from a single row of the original datachain representing a single paper we will get a new dataset with multiple rows per paper, each representing a single chunk).

We then specify what each row should contain by defining the `record` dictionary and then we use `yield Chunk(**record)` to create the new rows for each input row.

In [8]:
dc_chunks_embeddings = (
    dc_papers
    .limit(20) # we limit ourselves to 20 papers here, to speed up the demo
    .gen(document=pdf_chunks)
)

dc_chunks_embeddings.save("embeddings")

Processed: 738 rows [00:00, 11975.01 rows/s]
Processed: 0 rows [00:00, ? rows/s]
Processed: 2 rows [00:03,  1.68s/ rows]
Processed: 3 rows [00:06,  2.13s/ rows]
Processed: 4 rows [00:09,  2.51s/ rows]
Processed: 5 rows [00:13,  3.23s/ rows]
Processed: 6 rows [00:17,  3.47s/ rows]
Processed: 7 rows [00:20,  3.37s/ rows]
Processed: 8 rows [00:24,  3.54s/ rows]
Processed: 9 rows [00:28,  3.63s/ rows]
Processed: 10 rows [00:32,  3.64s/ rows]
Processed: 11 rows [00:36,  3.91s/ rows]
Processed: 12 rows [00:39,  3.66s/ rows]
Processed: 13 rows [00:43,  3.72s/ rows]
Processed: 14 rows [00:47,  3.76s/ rows]
Processed: 15 rows [00:50,  3.60s/ rows]
Processed: 16 rows [00:55,  3.91s/ rows]
Processed: 17 rows [00:58,  3.63s/ rows]
Processed: 18 rows [01:01,  3.45s/ rows]
Processed: 19 rows [01:05,  3.57s/ rows]
Processed: 20 rows [01:09,  3.70s/ rows]
Download: 51.2MB [01:13, 731kB/s]
Processed: 20 rows [01:13,  3.69s/ rows]
Generated: 3767 rows [01:10, 53.41 rows/s]


In the cell above we apply our new `pdf_chunks` function to the DataChain `dc_papers`. We do that by using the `gen` method of DataChain with `pdf_chunks`as its parameter. 

`DataChain.gen` is used when we have a function that creates multiple rows per single row of the original datachain (like in our examples, where each paper is split into multiple chunks)

We also presisted the result by the `.save` method. This will permanently save and version the datachain as a dataset with the name `embeddings`. Whenever we call `.save("embeddings")` again, a new version of this dataset will be saved automatically, so we can recall previous versions and track changes of the dataset over time.

### Evaluation

We will now use DataChain to calculate similarity between the two alternative embeddings using a fixed test query as reference and for further evaluation we will save dataset containing the chunks where the two embeddings differ the most.


In [9]:
TEST_QUERY = "What are the most promising approaches for combining neural networks with symbolic reasoning, according to recent NeurIPS papers?"

embedded_query_new = embedding_encoder_new.embed_query(query = TEST_QUERY)
embedded_query_old = embedding_encoder_old.embed_query(query = TEST_QUERY)

Using the built-in DataChain function `cosine_distance` we will calculate the cosine similarities between each chunk and the test query `TEST_QUERY` and then compare the results between the two embeddings.

To specify that we want to compare columns we use the `C` class from `datachain.lib.dc`. We use the `mutate` method of DataChain, which is a way to add new columns to an existing dataset.

Since we saved our dataset `embeddings`, we can now load its content to datachain by the `from_dataset` method

In [10]:
from datachain.sql.types import Float
from sqlalchemy import cast



embeddings_differences = (
    DataChain
    .from_dataset("embeddings")
    .mutate(
        query_sim_new = 1 - cosine_distance(C.document.embeddings_new, embedded_query_new),
        query_sim_old = 1 - cosine_distance(C.document.embeddings_old, embedded_query_old),
        )
    .mutate(abs_difference = cast(func.abs(C.query_sim_old - C.query_sim_new), Float))
    .filter(C.abs_difference > 0.1)
    .order_by("abs_difference", descending=True)
    .save("embeddings-differences")
)



In [11]:
embeddings_differences.show(20)

,document,document,document,document,query_sim_new,query_sim_old,abs_difference
,key,text,embeddings_new,embeddings_old,,,
0,072b030ba126b2f4b2374f342be9ed44,definition of information capacity for neural ...,"[-0.0007434347062371671, -0.033391062170267105...","[-0.03748973831534386, -0.07128849625587463, -...",0.599372,0.232001,0.367372
1,32bb90e8976aab5298d5da10fe66f21d,value a unit takes on after probing becomes de...,"[-0.029339516535401344, -0.008152814581990242,...","[-0.04788150265812874, -0.016090208664536476, ...",0.341957,-0.018574,0.360531
2,03afdbd66e7929b125f8597834fa83a4,"We expand E (d(vt, V2)) as follows","[0.019277652725577354, 0.005147224757820368, 0...","[0.026867331936955452, 0.1133461445569992, 0.1...",0.307695,-0.025924,0.333619
3,19ca14e7ea6328a42e0eb13d585e4c22,Following the method of analysis advanced in6 ...,"[-0.1627332717180252, -0.020131690427660942, -...","[-0.05186143517494202, 0.01539693120867014, -0...",0.282878,-0.025713,0.308591
4,03afdbd66e7929b125f8597834fa83a4,Substituting this estimate in the expression f...,"[-0.009833479300141335, 0.04947531223297119, 0...","[0.008176444098353386, 0.11296440660953522, 0....",0.252918,-0.054933,0.307851
5,072b030ba126b2f4b2374f342be9ed44,E(X) =- - ~ ~ W··X·X· + ~ t·X· I,"[0.03537482023239136, 0.012042378075420856, 0....","[-0.021044446155428886, 0.04361243546009064, 0...",0.247662,-0.058594,0.306256
6,03afdbd66e7929b125f8597834fa83a4,"To evaluate E(ln - h2- K I), we estimate the v...","[-0.015968868508934975, -0.04030385613441467, ...","[0.032369714230298996, 0.041351232677698135, 0...",0.222286,-0.081749,0.304036
7,1f0e3dad99908345f7439f8ffabdffc4,"[3] Brody D.A., IEEE Trans. vBME-32, n2, pl06-...","[-0.1226053535938263, 0.07194244116544724, -0....","[-0.11780540645122528, -0.06241508573293686, -...",0.043594,0.339062,0.295468
8,14bfa6bb14875e45bba028a21ed38046,We have studied the behaviour of the above mod...,"[-0.09533101320266724, 0.04399219900369644, 0....","[-0.0024363314732909203, -0.033086422830820084...",0.342260,0.047467,0.294794



[Limited by 20 rows]


We have now solved our scalability issues. When using `DataChain` locally, our computation will still be restricted to a simgle machine but for larger datasets you can use the SaaS version of DataChain available through our DVC Studio which comes with automatic computation cluster management, a graphical user interface and additional ML and data versioning features.

We have also solved our versioning needs and we can track the differences between embeddings over time and use that to choose the best embedding for our use-case.

### Adding more context - merging datasets

In our example bucket, we have not only the `pdf` files themselves but also additinal metadata stored as JSON files. We will now see how we can use Datachain to add the information about authors and the paper title to our `embeddings-differences` dataset which can help us with our evaluation.

In [12]:
import json
meta = json.load(open("sample.json"))
meta["authors"]

[{'given_name': 'Alan', 'family_name': 'Murray', 'institution': None},
 {'given_name': 'Anthony', 'family_name': 'Smith', 'institution': None},
 {'given_name': 'Zoe', 'family_name': 'Butler', 'institution': None}]

As we can see, the metadata contains information about the authors of the paper as a list of dictionaries with each author's name ane institution. Some values can be also be empty. Just as above we create a `DataModel` class to specify the outputs, keeping the name of the file as a key which we will use to join with the previous dataset.

Then we create a function to parse all this information and create a new dataset. We will now only create a single row per original dataset, so we use `return` instead of yield (and there is no need for the `Iterator` class)

In [13]:
# Define the output as a Feature class
class Metadata(DataModel):
    key: str
    title: str
    authors: list[dict[str, Optional[str]]]


# Use signatures to define input/output types (these can be Feature or regular Python types)
def extract_metadata(file: File) -> Metadata:
    import json
    # Ingest the file
    metadata = json.loads(file.read())

    record = dict()
    record["key"] = file.name.removesuffix("Metadata.json")+"Paper.pdf"
    record["title"] = metadata["title"]
    record["authors"] = metadata["authors"]

    return Metadata(**record)

We now apply the `extract_metadata` function as we did with `pdf_chunks` above, except that we use the `.map` method of DataChain which is employed when there is a 1:1 correspondence between the number of rows of the orignal and the new dataset.

In [23]:
dc_meta = (
    DataChain.from_storage("gs://datachain-demo/neurips")
    .filter(C.name.glob("*Metadata.json"))
    .map(document=extract_metadata)
    .select(
        "document.key",
        "document.title",
        "document.authors",
    )
    .save()
)

Processed: 738 rows [00:00, 12180.99 rows/s]
Download: 4.01MB [01:53, 37.1kB/s]
Processed: 184 rows [01:53,  1.63 rows/s]


Finally we will merge the two datasets using the DataChain `.merge` method.

In [24]:
dc_meta.show(2)

,document,document,document
,key,title,authors
0,02e74f10e0327ad868d138f2b4fdd6f0-Paper.pdf,Bit-Serial Neural Networks,"[{'given_name': 'Alan', 'family_name': 'Murray..."
1,03afdbd66e7929b125f8597834fa83a4-Paper.pdf,Connectivity Versus Entropy,"[{'given_name': 'Yaser', 'family_name': 'Abu-M..."



[Limited by 2 rows]


In [25]:
DataChain.from_dataset("embeddings-differences").merge(dc_meta, on="document.key").show(20)

,document,document,document,document,query_sim_new,query_sim_old,abs_difference,document,document
,key,text,embeddings_new,embeddings_old,,,,title,authors
0,072b030ba126b2f4b2374f342be9ed44,definition of information capacity for neural ...,"[-0.0007434347062371671, -0.033391062170267105...","[-0.03748973831534386, -0.07128849625587463, -...",0.599372,0.232001,0.367372,None,None
1,32bb90e8976aab5298d5da10fe66f21d,value a unit takes on after probing becomes de...,"[-0.029339516535401344, -0.008152814581990242,...","[-0.04788150265812874, -0.016090208664536476, ...",0.341957,-0.018574,0.360531,None,None
2,03afdbd66e7929b125f8597834fa83a4,"We expand E (d(vt, V2)) as follows","[0.019277652725577354, 0.005147224757820368, 0...","[0.026867331936955452, 0.1133461445569992, 0.1...",0.307695,-0.025924,0.333619,None,None
3,19ca14e7ea6328a42e0eb13d585e4c22,Following the method of analysis advanced in6 ...,"[-0.1627332717180252, -0.020131690427660942, -...","[-0.05186143517494202, 0.01539693120867014, -0...",0.282878,-0.025713,0.308591,None,None
4,03afdbd66e7929b125f8597834fa83a4,Substituting this estimate in the expression f...,"[-0.009833479300141335, 0.04947531223297119, 0...","[0.008176444098353386, 0.11296440660953522, 0....",0.252918,-0.054933,0.307851,None,None
5,072b030ba126b2f4b2374f342be9ed44,E(X) =- - ~ ~ W··X·X· + ~ t·X· I,"[0.03537482023239136, 0.012042378075420856, 0....","[-0.021044446155428886, 0.04361243546009064, 0...",0.247662,-0.058594,0.306256,None,None
6,03afdbd66e7929b125f8597834fa83a4,"To evaluate E(ln - h2- K I), we estimate the v...","[-0.015968868508934975, -0.04030385613441467, ...","[0.032369714230298996, 0.041351232677698135, 0...",0.222286,-0.081749,0.304036,None,None
7,1f0e3dad99908345f7439f8ffabdffc4,"[3] Brody D.A., IEEE Trans. vBME-32, n2, pl06-...","[-0.1226053535938263, 0.07194244116544724, -0....","[-0.11780540645122528, -0.06241508573293686, -...",0.043594,0.339062,0.295468,None,None
8,14bfa6bb14875e45bba028a21ed38046,We have studied the behaviour of the above mod...,"[-0.09533101320266724, 0.04399219900369644, 0....","[-0.0024363314732909203, -0.033086422830820084...",0.342260,0.047467,0.294794,None,None



[Limited by 20 rows]
